## Acessar o Dataset e Imports


### Imports


In [ ]:
!pip install librosa scikit-learn matplotlib

In [ ]:
!pip install jiwer
!pip install num2words

In [ ]:
!pip install pydub

In [ ]:
!pip install SpeechRecognition

In [ ]:
!pip install num2words

In [ ]:
!pip install torchaudio

In [ ]:
!pip install transformers

In [ ]:
import glob
import os
import librosa
import numpy as np
import torchaudio
import torch
from torchaudio.transforms import Resample
from IPython import display as disp
from jiwer import wer
from IPython.display import Audio

In [ ]:
from num2words import num2words
import re
from decimal import Decimal, InvalidOperation

###Dataset
Vamos importar o nosso CSV, para usa-lo em análises futuras e também implementar a análise juntamente com os áudios separados em uma pasta.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd

# Aqui você pode substituir com o caminho para o seu arquivo CSV
csv_file = "/content/DataSets/dataset/metadata.csv"


# Carregar o dataset
dataset = pd.read_csv(csv_file)


In [ ]:
pd.DataFrame(dataset["audio_segmentado"])

In [ ]:
# Removemos colunas que não serão utilizadas
colunas_para_remover = ['locutor', 'sentimento','sotaque'] # Exemplos de colunas
dataset = dataset.drop(colunas_para_remover, axis=1)

In [ ]:
contagem_valores = dataset['nivel_ruido'].value_counts()

# Filtrando apenas os valores que se repetem (aparecem mais de uma vez)
valores_repetidos = contagem_valores[contagem_valores > 1]

# Imprimindo os valores que se repetem
print(valores_repetidos)

# RMS + WER + Model

Calcular a Amplitude RMS: RMS (Root Mean Square) é um método comum para medir a amplitude do sinal de áudio, que pode ser usado para estimar o nível de ruído. A amplitude RMS é calculada como a raiz quadrada da média dos quadrados das amplitudes.



Converter para Decibéis: Para converter a amplitude RMS para decibéis, usamos a fórmula 20 * log10(amplitude).



## Transcrição humana e sua normalização
Temos a transcrição humana já anotada no dataset, por isso precisamos do CSV.


In [ ]:
def coleta_transcricao_humana(id_audio):
    # Acessar a linha específica utilizando o ID do audio (assumindo que 'audio_segmentado' é a coluna com esses IDs)
    linha = dataset[dataset['audio_segmentado'] == id_audio]

    # Acessando a transcrição humana associada a esse segmento de áudio
    transcricao_humana = linha['transcricao_normalizada'].values[0] if not linha.empty else "Transcrição não encontrada."

    # print(f"Transcrição Humana Normalizada: {transcricao_humana}")
    return transcricao_humana

## Transcrição obtida normalização
Vamos normalizar as transcrições obtidas pelo modelo.


### Normalizar áudio

####Converter numero por extenso


In [ ]:
# Função para converter números em extenso
def converter_em_extenso(input):
    # Verifica se o input é um número válido
    try:
        # Tentativa de conversão para Decimal para garantir que o valor é numérico
        num = Decimal(re.sub(r'[^\d.]', '', input))
        return num2words(num, lang='pt_BR')
    except InvalidOperation:
        # Se não for possível converter, retorna o input original ou uma mensagem de erro
        return input  # ou "valor inválido"

# Função atualizada para verificar e extrair números
def verificar_numero(frase):
    numeros = re.findall(r'\d+', frase)
    if numeros:
        return numeros[0]  # Retorna o primeiro número encontrado
    return None  # Retorna None se nenhum número for encontrado

# Função para substituir números na frase por sua representação por extenso
def substituir_numeros_por_extenso(frase):
    numeros = re.findall(r'\d+', frase)
    for num in numeros:
        frase = frase.replace(num, converter_em_extenso(num), 1)
    return frase

####Normalização do texto


In [ ]:
# Função de normalização atualizada para incluir a substituição de números
def normalize_text(text):
    text = text.lower()  # Converter para minúsculas
    text = re.sub(r'[^\w\s]', '', text)  # Remover pontuação
    text = re.sub(r'\s+', ' ', text).strip()  # Remover espaços extras
    if verificar_numero(text):  # Se houver números, substitua por extenso
        text = substituir_numeros_por_extenso(text)
    return text

##Modelo
Vamos então usar um modelo que seja eficiente para transcrever os áudios. O modelo fica a seu critério, usamos aqui o Distill Whisper do Hugging Face.

In [ ]:
# Aqui você pode adicionar o seu modelo de transcrição speech to text
# Optamos pelo modelo Whisper, fica a sua escolha.
lang = "pt"
model_name = "pierreguillou/whisper-medium-portuguese"

In [ ]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# carregamos e processamos o modelo
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)

In [ ]:
import torch
import librosa

# Aqui fazemos um teste para saber se o modelo está rodando na GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

# A função que realiza a inferência, normaliza a transcrição e substitui números por extenso
def transcribe_distil_whisper(audio_path):
    # Carregar o áudio
    raw_audio, _ = librosa.load(audio_path, sr=16000)

    # Processar o áudio para criar os tensores de entrada e enviar para a GPU
    inputs = processor(raw_audio, sampling_rate=16000, return_tensors="pt").input_features.to(device)

    # Colocar o modelo em modo de avaliação e enviá-lo para a GPU
    model.eval().to(device)

    # Realizar a inferência
    with torch.no_grad():
        generated_ids = model.generate(inputs)

    # Decodificar a transcrição e normalizá-la
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    normalized_transcription = normalize_text(transcription)
    return normalized_transcription


##Calculo do RMS
Calculamos o RMS e também criamos listas para armazenar os valores gerados pela análise.


In [ ]:
# Inicializando listas para coletar dados de nível de ruído e WER
niveis_de_ruido = []
classificacoes_de_ruido = []
erros_distil_list = []


In [ ]:
# Criamos algumas funções para análisar o audio.
# Primeiro calculamos o nível de ruido atráves do RMS
def calcular_nivel_de_ruido(caminho_do_audio):
    audio, sr = librosa.load(caminho_do_audio)
    # Reproduzir o áudio
    rms = np.sqrt(np.mean(audio**2))
    nivel_de_ruido_db = 20 * np.log10(rms) if rms > 0 else -np.inf
    return nivel_de_ruido_db

# Depois criamos outra função para verificar o nível de wer e distribuir sua classificação com base nos valores
def classificar_nivel_de_ruido(nivel_de_ruido, erro_distil):
    if erro_distil <= 0.36:
        print("Wer = ", erro_distil)
        return "Nenhum"
    elif erro_distil <= 0.45:
        print("Wer = ", erro_distil)
        return "Pouco"
    elif erro_distil > 0.45 and erro_distil < 0.70:
        print("Wer = ", erro_distil)
        return "Médio"
    else:
        print("Wer = ", erro_distil)
        return "Muito"


## Calculando o WER

"Word Error Rate" (Taxa de Erro de Palavras), é uma métrica usada para avaliar a precisão de sistemas de reconhecimento de voz, como áudios no nosso caso. Ela calcula a proporção de palavras incorretamente reconhecidas em comparação com uma transcrição de referência (Transcrição Humana), considerando inserções, deleções e substituições de palavras. É expressa como uma porcentagem, onde valores menores indicam melhor desempenho.

Usamos o WER para entender se o nosso modelo está se comportando bem na sua análise.

In [ ]:
# Criamos uma função para calcular o WER
# com base na transcrição humana e transcrição obtida pelo modelo
def calcular_wer(transcricao_referencia, transcribe_obtida_distil_whisper):
    print(f"{transcricao_referencia}\n----\n{transcribe_obtida_distil_whisper}\n----\n")
    erro_distil = wer(transcricao_referencia, transcribe_obtida_distil_whisper)
    return erro_distil

In [ ]:
def analisar_audio_e_transcricao(caminho_do_audio, transcricao_referencia, transcribe_obtida_distil_whisper):
    # Funções calcular_nivel_de_ruido e classificar_nivel_de_ruido precisam estar definidas
    nivel_de_ruido = calcular_nivel_de_ruido(caminho_do_audio)
    niveis_de_ruido.append(nivel_de_ruido)
    # Chama calcular_wer com todos os argumentos corretamente para cada par de transcrições
    erro_palavras_distil = calcular_wer(transcricao_referencia, transcribe_obtida_distil_whisper)
    classificacao_ruido = classificar_nivel_de_ruido(nivel_de_ruido, erro_palavras_distil)
    classificacoes_de_ruido.append(classificacao_ruido)
    erros_distil_list.append(erro_palavras_distil)

    # Imprime resultados
    print(f"Nível de Ruído em dB: {nivel_de_ruido:.2f} dB")
    print(f"Classificação do Nível de Ruído: {classificacao_ruido}")

    # Retorna um dicionário com os resultados
    return {
        'erro_palavras_distil': erro_palavras_distil,
        'classificacao_ruido': classificacao_ruido,
        'nivel_de_ruido': nivel_de_ruido
    }

# Classificação dos ruidos
Aqui é onde chamamos todas as funções anteriores e fazemos a classificação, assim que cada áudio é processado, o algoritmo vai criar um dataset novo e depois vai atualizando com as novas informações atualizadas.

In [ ]:
import os
import pandas as pd
from speech_recognition import UnknownValueError

# Função para verificar se um arquivo existe no caminho especificado
def verificar_existencia_arquivo(caminho):
    return os.path.exists(caminho)

# Inicializa a contagem de áudios processados
contagem_processados = 0

# Adiciona novas colunas ao DataFrame
dataset['taxa_de_wer'] = None
dataset['nivel_ruido_distill'] = None
dataset['niveis_de_ruido_dB'] = None

# Itera sobre as primeiras 10 linhas do DataFrame
for index, row in dataset.iloc[0:10].iterrows():
    # Obtém o ID do áudio segmentado da linha atual
    id_audio = row['audio_segmentado']
    # Constroi o caminho completo do arquivo de áudio
    caminho_do_audio = f'/content/DataSets/dataset/audios/{id_audio}'
    print(caminho_do_audio)

    # Verifica se o arquivo de áudio existe
    if verificar_existencia_arquivo(caminho_do_audio):
        # Coleta a transcrição humana de referência
        transcricao_referencia = coleta_transcricao_humana(id_audio).lower()

        try:
            # Obtém a transcrição usando a função transcribe_distil_whisper
            transcribe_obtida_distil_whisper = transcribe_distil_whisper(caminho_do_audio).lower()
            # Analisa o áudio e a transcrição obtida
            resultado_analise = analisar_audio_e_transcricao(caminho_do_audio, transcricao_referencia, transcribe_obtida_distil_whisper)
            # Atualiza o DataFrame com os resultados da análise
            dataset.at[index, 'taxa_de_wer'] = resultado_analise['erro_palavras_distil']
            dataset.at[index, 'nivel_ruido_distill'] = resultado_analise['classificacao_ruido']
            dataset.at[index, 'niveis_de_ruido_dB'] = resultado_analise['nivel_de_ruido']
            # Incrementa a contagem de áudios processados
            contagem_processados += 1

        except UnknownValueError:
            # Tratamento de erro para quando a transcrição não é possível
            print(f"Não foi possível transcrever o áudio: {id_audio}")
            dataset.at[index, 'taxa_de_wer'] = 'Erro de transcrição'
            dataset.at[index, 'nivel_ruido_distill'] = 'Erro de transcrição'
            dataset.at[index, 'niveis_de_ruido_dB'] = 'Erro de transcrição'

    else:
        # Tratamento para quando o arquivo de áudio não é encontrado
        print(f"Arquivo não encontrado: {id_audio}")
        dataset.at[index, 'taxa_de_wer'] = 'Arquivo não encontrado'
        dataset.at[index, 'nivel_ruido_distill'] = 'Arquivo não encontrado'
        dataset.at[index, 'niveis_de_ruido_dB'] = 'Arquivo não encontrado'

    # Exibe a contagem de áudios processados até o momento
    print(f"Áudios processados: {contagem_processados}/{len(dataset) - 1}")
    # Salva o DataFrame atualizado em um arquivo CSV a cada iteração para evitar perda de dados
    dataset.to_csv('dataset_atualizado.csv', index=False)

# Indica o término da análise de WER
print("Finalizado toda análise de WER.")
